In [1]:
import pandas as pd  
import numpy as np
import torch
from chronos import BaseChronosPipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tqdm as notebook_tqdm

In [2]:
# Using GPU if available
device_map = "cpu"
device = torch.device("cpu")

In [3]:
# Load Chronos model
pipeline = BaseChronosPipeline.from_pretrained(
    "amazon/chronos-bolt-small",  # or "chronos-t5-small"
    device_map=device,
    torch_dtype=torch.float32
)

In [4]:
# Constants
ID_COL = "PERMNO"
TARGET_COL = "ExcessReturn"
DATE_COL = "DlyCalDt"
WINDOW = 5  # days of context

# Load data
in_sample_df = pd.read_csv("../Cleaned Datasets/in_sample_cleaned.csv")
out_sample_df = pd.read_csv("../Cleaned Datasets/out_sample_cleaned.csv")

In [5]:
in_sample_df = in_sample_df[['PERMNO', 'DlyCalDt', 'ExcessReturn']]
out_sample_df = out_sample_df[['PERMNO', 'DlyCalDt', 'ExcessReturn']]

print(in_sample_df.info())
print(out_sample_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199550 entries, 0 to 199549
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   PERMNO        199550 non-null  int64  
 1   DlyCalDt      199550 non-null  object 
 2   ExcessReturn  199550 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 4.6+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112400 entries, 0 to 112399
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   PERMNO        112400 non-null  int64  
 1   DlyCalDt      112400 non-null  object 
 2   ExcessReturn  112400 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.6+ MB
None


In [18]:
# Create rolling window for predictors for Bolt models

combined_df = pd.concat([in_sample_df, out_sample_df])
combined_df = combined_df.sort_values([ID_COL, DATE_COL]).reset_index(drop=True)
combined_df[DATE_COL] = pd.to_datetime(combined_df[DATE_COL])

contexts = []
targets = []
records = []

for id, grp in combined_df.groupby(ID_COL):
    values = grp[TARGET_COL].values
    dates = grp[DATE_COL].values

    for i in range(len(values) - WINDOW):
        pred_date = dates[i + WINDOW]
        if pred_date >= pd.to_datetime("2016-01-01"):
            contexts.append(torch.tensor(values[i:i+WINDOW], dtype=torch.float32, device=device))
            targets.append(values[i+WINDOW])
            records.append({
                ID_COL: id,
                TARGET_COL: values[i+WINDOW],
                DATE_COL: pred_date
            })


In [22]:
contexts

[tensor([-0.0041,  0.0037,  0.0052, -0.0138, -0.0083]),
 tensor([ 0.0037,  0.0052, -0.0138, -0.0083, -0.0182]),
 tensor([ 0.0052, -0.0138, -0.0083, -0.0182,  0.0041]),
 tensor([-0.0138, -0.0083, -0.0182,  0.0041, -0.0224]),
 tensor([-0.0083, -0.0182,  0.0041, -0.0224, -0.0267]),
 tensor([-0.0182,  0.0041, -0.0224, -0.0267, -0.0254]),
 tensor([ 0.0041, -0.0224, -0.0267, -0.0254, -0.0054]),
 tensor([-0.0224, -0.0267, -0.0254, -0.0054,  0.0261]),
 tensor([-0.0267, -0.0254, -0.0054,  0.0261, -0.0276]),
 tensor([-0.0254, -0.0054,  0.0261, -0.0276,  0.0109]),
 tensor([-0.0054,  0.0261, -0.0276,  0.0109, -0.0096]),
 tensor([ 0.0261, -0.0276,  0.0109, -0.0096,  0.0094]),
 tensor([-0.0276,  0.0109, -0.0096,  0.0094, -0.0072]),
 tensor([ 0.0109, -0.0096,  0.0094, -0.0072,  0.0072]),
 tensor([-0.0096,  0.0094, -0.0072,  0.0072,  0.0285]),
 tensor([ 0.0094, -0.0072,  0.0072,  0.0285, -0.0300]),
 tensor([-0.0072,  0.0072,  0.0285, -0.0300,  0.0189]),
 tensor([ 0.0072,  0.0285, -0.0300,  0.0189, -0.

In [24]:
combined_df

,PERMNO,DlyCalDt,ExcessReturn
0,10138,2000-01-03,-0.049215
1,10138,2000-01-04,-0.030396
2,10138,2000-01-05,-0.001983
3,10138,2000-01-06,0.029266
4,10138,2000-01-07,-0.007289
...,...,...,...
311945,92655,2024-12-24,-0.000587
311946,92655,2024-12-26,0.009863
311947,92655,2024-12-27,-0.002380
311948,92655,2024-12-30,-0.004409


In [19]:
print(len(contexts))

112400


In [8]:
y_test = pd.Series(targets)

results = pd.DataFrame(records)

In [9]:
# Creating a Function to Calculate Predictive-R2 Used in the Finance Literature
def r2(y_true, y_pred):
    return 1-(((y_true-y_pred)**2).sum()/(y_true**2).sum())

In [10]:
# Zero Shot Chronos-Bolt-Tiny
pipeline = BaseChronosPipeline.from_pretrained(
    'amazon/chronos-bolt-tiny',
    device_map=device_map,
    torch_dtype=torch.float32
)

quantiles, means = pipeline.predict_quantiles(context=contexts, prediction_length=1, quantile_levels=[0.5])
preds = [m.cpu().squeeze().item() for m in means]
y_chr_bolt_tiny = pd.Series(preds)

results['y_chr_bolt_tiny'] = y_chr_bolt_tiny
r2_chr_bolt_tiny  = r2(y_test, y_chr_bolt_tiny)
mse_chr_bolt_tiny = mean_squared_error(y_test, y_chr_bolt_tiny)
mae_chr_bolt_tiny = mean_absolute_error(y_test, y_chr_bolt_tiny)
da_chr_bolt_tiny = (np.sign(y_test) == np.sign(y_chr_bolt_tiny)).mean()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


In [11]:
# Zero Shot Chronos-Bolt-Mini
pipeline = BaseChronosPipeline.from_pretrained(
    'amazon/chronos-bolt-mini',
    device_map=device_map,
    torch_dtype=torch.float32
)

quantiles, means = pipeline.predict_quantiles(context=contexts, prediction_length=1, quantile_levels=[0.5])
preds = [m.cpu().squeeze().item() for m in means]
y_chr_bolt_mini = pd.Series(preds)

results['y_chr_bolt_mini'] = y_chr_bolt_mini
r2_chr_bolt_mini  = r2(y_test, y_chr_bolt_mini)
mse_chr_bolt_mini = mean_squared_error(y_test, y_chr_bolt_mini)
mae_chr_bolt_mini = mean_absolute_error(y_test, y_chr_bolt_mini)
da_chr_bolt_mini = (np.sign(y_test) == np.sign(y_chr_bolt_mini)).mean()

In [12]:
# Zero Shot Chronos-Bolt-Small
pipeline = BaseChronosPipeline.from_pretrained(
    'amazon/chronos-bolt-small',
    device_map=device_map,
    torch_dtype=torch.float32
)

preds = []
for ctx in contexts:
    quantiles, mean = pipeline.predict_quantiles(context=[ctx.to(device)], prediction_length=1, quantile_levels=[0.5])
    preds.append(mean.cpu().squeeze().item())
y_chr_bolt_small = pd.Series(preds)

results['y_chr_bolt_small'] = y_chr_bolt_small
r2_chr_bolt_small  = r2(y_test, y_chr_bolt_small)
mse_chr_bolt_small = mean_squared_error(y_test, y_chr_bolt_small)
mae_chr_bolt_small = mean_absolute_error(y_test, y_chr_bolt_small)
da_chr_bolt_small = (np.sign(y_test) == np.sign(y_chr_bolt_small)).mean()

KeyboardInterrupt: 

In [ ]:
# Zero Shot Chronos-Bolt-Base
pipeline = BaseChronosPipeline.from_pretrained(
    'amazon/chronos-bolt-base',
    device_map=device_map,
    torch_dtype=torch.float32
)

preds = []
for ctx in contexts:
    quantiles, mean = pipeline.predict_quantiles(context=[ctx.to(device)], prediction_length=1, quantile_levels=[0.5])
    preds.append(mean.cpu().squeeze().item())
y_chr_bolt_base = pd.Series(preds)

results['y_chr_bolt_base'] = y_chr_bolt_base
r2_chr_bolt_base  = r2(y_test, y_chr_bolt_base)
mse_chr_bolt_base = mean_squared_error(y_test, y_chr_bolt_base)
mae_chr_bolt_base = mean_absolute_error(y_test, y_chr_bolt_base)
da_chr_bolt_base = (np.sign(y_test) == np.sign(y_chr_bolt_base)).mean()

C:\Users\amaan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\amaan\.cache\huggingface\hub\models--amazon--chronos-bolt-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:
# Switching to using CPU for T5 models
device_map = "cpu"
device = torch.device("cpu")

In [ ]:
# Create rolling window for predictors for T5 models
contexts = []
targets = []

for id, grp in combined_df.groupby(ID_COL):
    values = grp[TARGET_COL].values
    dates = grp[DATE_COL].values

    for i in range(len(values) - WINDOW):
        pred_date = dates[i + WINDOW]
        if pred_date >= pd.to_datetime("2016-01-01"):
            contexts.append(torch.tensor(values[i:i+WINDOW], dtype=torch.float32, device=device))
            targets.append(values[i+WINDOW])

In [ ]:
# Zero Shot Chronos-T5-Tiny
pipeline = BaseChronosPipeline.from_pretrained(
    'amazon/chronos-t5-tiny',
    device_map=device_map,
    torch_dtype=torch.float32
)

preds = []
for ctx in contexts:
    quantiles, mean = pipeline.predict_quantiles(context=[ctx], prediction_length=1, quantile_levels=[0.5])
    preds.append(mean.squeeze().item())
y_chr_t5_tiny = pd.Series(preds)

results['y_chr_t5_tiny'] = y_chr_t5_tiny
r2_chr_t5_tiny  = r2(y_test, y_chr_t5_tiny)
mse_chr_t5_tiny = mean_squared_error(y_test, y_chr_t5_tiny)
mae_chr_t5_tiny = mean_absolute_error(y_test, y_chr_t5_tiny)
da_chr_t5_tiny = (np.sign(y_test) == np.sign(y_chr_t5_tiny)).mean()

In [ ]:
# Zero Shot Chronos-T5-Mini
pipeline = BaseChronosPipeline.from_pretrained(
    'amazon/chronos-t5-mini',
    device_map=device_map,
    torch_dtype=torch.float32
)

preds = []
for ctx in contexts:
    quantiles, mean = pipeline.predict_quantiles(context=[ctx], prediction_length=1, quantile_levels=[0.5])
    preds.append(mean.squeeze().item())
y_chr_t5_mini = pd.Series(preds)

results['y_chr_t5_mini'] = y_chr_t5_mini
r2_chr_t5_mini  = r2(y_test, y_chr_t5_mini)
mse_chr_t5_mini = mean_squared_error(y_test, y_chr_t5_mini)
mae_chr_t5_mini = mean_absolute_error(y_test, y_chr_t5_mini)
da_chr_t5_mini = (np.sign(y_test) == np.sign(y_chr_t5_mini)).mean()

In [ ]:
# Zero Shot Chronos-T5-Small
pipeline = BaseChronosPipeline.from_pretrained(
    'amazon/chronos-t5-small',
    device_map=device_map,
    torch_dtype=torch.float32
)

preds = []
for ctx in contexts:
    quantiles, mean = pipeline.predict_quantiles(context=[ctx], prediction_length=1, quantile_levels=[0.5])
    preds.append(mean.squeeze().item())
y_chr_t5_small = pd.Series(preds)

results['y_chr_t5_small'] = y_chr_t5_small
r2_chr_t5_small  = r2(y_test, y_chr_t5_small)
mse_chr_t5_small = mean_squared_error(y_test, y_chr_t5_small)
mae_chr_t5_small = mean_absolute_error(y_test, y_chr_t5_small)
da_chr_t5_small = (np.sign(y_test) == np.sign(y_chr_t5_small)).mean()

In [ ]:
# Collating Results

results_matrix = [{
        "Model": "Chronos-Bolt-Tiny",
        "R-squared": r2_chr_bolt_tiny,
        "MSE": mse_chr_bolt_tiny,
        "MAE": mae_chr_bolt_tiny,
        "Direction Accuracy": da_chr_bolt_tiny
    },
    {
        "Model": "Chronos-Bolt-Mini",
        "R-squared": r2_chr_bolt_mini,
        "MSE": mse_chr_bolt_mini,
        "MAE": mae_chr_bolt_mini,
        "Direction Accuracy": da_chr_bolt_mini
    },
    {
        "Model": "Chronos-Bolt-Small",
        "R-squared": r2_chr_bolt_small,
        "MSE": mse_chr_bolt_small,
        "MAE": mae_chr_bolt_small,
        "Direction Accuracy": da_chr_bolt_small
    },
    {
        "Model": "Chronos-Bolt-Base",
        "R-squared": r2_chr_bolt_base,
        "MSE": mse_chr_bolt_base,
        "MAE": mae_chr_bolt_base,
        "Direction Accuracy": da_chr_bolt_base
    },
    {
        "Model": "Chronos-T5-Tiny",
        "R-squared": r2_chr_t5_tiny,
        "MSE": mse_chr_t5_tiny,
        "MAE": mae_chr_t5_tiny,
        "Direction Accuracy": da_chr_t5_tiny
    },
    {
        "Model": "Chronos-T5-Mini",
        "R-squared": r2_chr_t5_mini,
        "MSE": mse_chr_t5_mini,
        "MAE": mae_chr_t5_mini,
        "Direction Accuracy": da_chr_t5_mini
    },
    {
        "Model": "Chronos-T5-Small",
        "R-squared": r2_chr_t5_small,
        "MSE": mse_chr_t5_small,
        "MAE": mae_chr_t5_small,
        "Direction Accuracy": da_chr_t5_small
    }]

results_matrix_df = pd.DataFrame(results_matrix)
results_matrix_df

In [ ]:
# Save Prediction Results
results.to_csv("chronos_predictions(5-day).csv", index=False)
results_matrix_df.to_csv("chronos_results(5-day).csv", index=False)